# Practica 4

In [15]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances_argmin_min
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances

import warnings

# Desactivar los warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('./Dataset_lab04/PUBG.csv')
df

player_name  tracker_id  solo_KillDeathRatio  solo_WinRatio  \
0         BreakNeck        4405                 3.14          17.65   
1         Blackwalk        8199                 4.41          18.18   
2     mercedes_benz        4454                 3.60           0.00   
3              DORA        7729                14.00          50.00   
4           n2tstar           0                10.50          33.33   
..              ...         ...                  ...            ...   
94         jungwook       11690                 3.37          17.33   
95           Sitton        6042                 6.00          27.27   
96      PANDATV-TYF       22188                 3.05           9.84   
97  Great_Cornholio        2260                 0.00           0.00   
98            Cafri        9346                 3.10          16.67   

    solo_TimeSurvived  solo_RoundsPlayed  solo_Wins  solo_WinTop10Ratio  \
0            18469.14                 17          3                0.83   
1            33014.86                 33          6                0.36   
2             4330.44                  5          0                0.00   
3            13421.82                  8          4                0.67   
4             9841.04                  6          2                0.40   
..                ...                ...        ...                 ...   
94          581263.40                427         74                0.38   
95           15639.28                 11          3                0.60   
96          243659.10                193         19                0.25   
97             561.04                  1          0                0.00   
98           17552.90                 12          2                0.40   

    solo_Top10s  solo_Top10Ratio  ...  squad_RideDistance  squad_MoveDistance  \
0             4             23.5  ...          3751590.99          5194786.58   
1            11             33.3  ...          4295917.30          6051783.67   
2             1             20.0  ...          3935265.63          5589608.74   
3             6             75.0  ...          2738998.00          3796916.00   
4             5             83.3  ...          2347295.00          3220260.00   
..          ...              ...  ...                 ...                 ...   
94          195             45.7  ...           585880.94          1010687.19   
95            5             45.5  ...          1370085.00          2075036.00   
96           77             39.9  ...          1079497.60          1870113.40   
97            0              0.0  ...          1280647.63          2002011.90   
98            5             41.7  ...          3460731.00          4717473.00   

    squad_AvgWalkDistance  squad_AvgRideDistance  squad_LongestKill  \
0                 2626.97                4372.64             536.98   
1                 2422.48                6009.73             734.85   
2                 1871.89                3011.87             725.44   
3                 2154.62                5578.41             587.28   
4                 2098.47                5642.54             546.10   
..                    ...                    ...                ...   
94                2015.37                2582.05             796.52   
95                2049.28                3982.80             511.08   
96                1750.42                2107.95             560.26   
97                1848.11                1898.83             730.65   
98                1615.35                4448.24             370.99   

    squad_Heals  squad_Revives  squad_Boosts  squad_DamageDealt  squad_DBNOs  
0          2186            234          1884          242132.73         1448  
1          2041            276          2340          269795.75         1724  
2          1766            210          2193          292977.07         1897  
3          1214            142          1252          181106.90         1057  
4          1245            120   

In [18]:

# Seleccionar columnas relevantes para el clustering
columns = ['solo_KillDeathRatio', 'solo_WinRatio', 'solo_TimeSurvived', 'solo_RoundsPlayed']

# Convertir columnas relevantes a tipo numérico
df[columns] = df[columns].apply(pd.to_numeric, errors='coerce')

# Eliminar filas con valores no numéricos
df = df.dropna(subset=columns)

# Preprocesamiento de datos
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[columns])

# Función para calcular las distancias y asignar las etiquetas
def assign_labels(data, centroids):
    distances = pairwise_distances(data, centroids)
    labels = np.argmin(distances, axis=1)
    return labels

# Función para ejecutar el algoritmo de k-means con diferentes configuraciones
def run_kmeans(data, n_clusters, metric):
    kmeans = KMeans(n_clusters=n_clusters, init='random', random_state=42, algorithm='full')
    kmeans.fit(data)
    kmeans.labels_ = assign_labels(data, kmeans.cluster_centers_)
    kmeans.inertia_ = kmeans.inertia_
    return kmeans

# Configuraciones a probar
num_clusters = [2, 3, 4, 5]
metrics = ['euclidean', 'cosine']

# Tabla para almacenar los resultados
results = pd.DataFrame(columns=['Configuración', 'Iteraciones'])

# Ejecutar el algoritmo para diferentes configuraciones
for n_clusters in num_clusters:
    for metric in metrics:
        kmeans = run_kmeans(df_scaled, n_clusters, metric)
        results = results.append({'Configuración': f"{n_clusters} grupos, {metric} distancia",
                                  'Iteraciones': kmeans.n_iter_}, ignore_index=True)

# Mostrar los resultados
print(results)

# Obtener la mejor configuración
best_config = results.loc[results['Iteraciones'].idxmin()]['Configuración']
print("Mejor configuración:", best_config)

# Ejemplo de visualización de grupos
best_kmeans = run_kmeans(df_scaled, n_clusters=4, metric='euclidean')
df['cluster'] = best_kmeans.labels_
plt.scatter(df['solo_KillDeathRatio'], df['solo_WinRatio'], c=df['cluster'])
plt.xlabel('solo_KillDeathRatio')
plt.ylabel('solo_WinRatio')
plt.show()

                   Configuración Iteraciones
0  2 grupos, euclidean distancia           3
1     2 grupos, cosine distancia           3
2  3 grupos, euclidean distancia           5
3     3 grupos, cosine distancia           5
4  4 grupos, euclidean distancia           7
5     4 grupos, cosine distancia           7
6  5 grupos, euclidean distancia           6
7     5 grupos, cosine distancia           6


TypeError: reduction operation 'argmin' not allowed for this dtype

In [22]:
from sklearn.cluster import KMeans

# Preparación de los datos (supongamos que tenemos una matriz de datos llamada "jugadores")

datos = pd.read_csv('./Dataset_lab04/PUBG.csv')
# Selección del número de grupos (k)
num_grupos = 3

# Creación del modelo k-means
kmeans = KMeans(n_clusters=num_grupos)

# Ajuste del modelo a los datos
kmeans.fit(datos)

# Obtención de las etiquetas de grupo asignadas a cada jugador
etiquetas = kmeans.labels

# Obtención de los centroides de los grupos
centroides = kmeans.clustercenters

# Número de iteraciones necesarias para converger
num_iteraciones = kmeans.niter

# Impresión de los resultados
for i in range(num_grupos):
    jugadores_grupo_i = jugadores[etiquetas == i]
    print(f"Grupo {i+1}:")
    print(jugadores_grupo_i)
    print("")

print(f"Número de iteraciones: {num_iteraciones}")

ValueError: could not convert string to float: 'BreakNeck'